In [1]:
1+1

2

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
load_dotenv()
import os

api_key=os.getenv("GAMINI_API_KEY")


In [3]:
model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b",
    api_key=api_key,
)

In [9]:
dsa_solver=AssistantAgent(
    name="dsa_solver",
    model_client=model_client,
    system_message="you give me code in c++ to solve complex dsa problem"
)

code_reviewer=AssistantAgent(
    name="recviewer",
    model_client=model_client,
    system_message="you review the code by the dsa_solver "
)
code_editer=AssistantAgent(
    name="code_editer",
    model_client=model_client,
    system_message="you make the code easy to understand and add comment of the code "
)

In [13]:
from autogen_agentchat.teams import RoundRobinGroupChat
team=RoundRobinGroupChat(
    participants=[dsa_solver,code_reviewer,code_editer],
    max_turns=3
)

In [14]:
from autogen_agentchat.messages import TextMessage

async def test_team():
    task=TextMessage(content="write code to of find prime number in c++",source='user')
    result= await team.run(task=task)

    for each_messages in result.messages:
        print(f'{each_messages.source}:{each_messages.content}')

In [15]:
await test_team()

user:write code to of find prime number in c++
dsa_solver:```cpp
#include <iostream>
#include <cmath>

using namespace std;

// Function to check if a number is prime
bool isPrime(int num) {
    // 1 and numbers less than 2 are not prime
    if (num <= 1) {
        return false;
    }
    // 2 is a prime number
    if (num <= 3) {
        return true;
    }

    // This optimization is crucial for efficiency.  We only need to check divisibility
    // up to the square root of the number.
    if (num % 2 == 0 || num % 3 == 0) {
        return false;
    }

    for (int i = 5; i * i <= num; i = i + 6) {
        if (num % i == 0 || num % (i + 2) == 0) {
            return false;
        }
    }

    return true;
}


int main() {
    int num;

    cout << "Enter a positive integer: ";
    cin >> num;

    if (num <= 0) {
        cout << "Please enter a positive integer." << endl;
        return 1;  // Indicate an error
    }


    if (isPrime(num)) {
        cout << num << " is a prime num